In [2]:
import spacy

In [3]:
nlp = spacy.load("en_core_web_sm")

OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

In [3]:
from tqdm import tqdm
import spacy
import random
import json
import re

In [4]:
training_data = []
lines=[]
with open('final_training_data.json', 'r') as f:
    lines = f.readlines()

for line in lines:
    data = json.loads(line)
    text = data['content']
    entities = []
    if(data['annotation'] is not None):
      for annotation in data['annotation']:
          if(annotation is not None):
            #only a single point in text annotation.
            point = annotation['points'][0]
            labels = annotation['label']
            # handle both list of labels or a single label.
            if not isinstance(labels, list):
                labels = [labels]

            for label in labels:
                #dataturks indices are both inclusive [start, end] but spacy is not [start, end)
                entities.append((point['start'], point['end'] + 1 ,label))

    if(entities != []):
        training_data.append((text, {"entities" : entities}))

In [5]:
len(training_data)

446

In [6]:
training_data[0:5]

[('Mrs. Shalini visited Dinanath Mangeshkar Hospital, Pune.,',
  {'entities': [(51, 55, 'GPE'), (21, 49, 'ORG'), (5, 12, 'PERSON')]}),
 ('Her physician was Dr Arun Wardhan and she was happy with results of ETT. ,',
  {'entities': [(21, 33, 'PERSON')]}),
 ("Manisha was in Cardiac rehab locally at Indore last week and BP 170/80.  They called us and we increased her HCTZ to 25 mg from 12.5 mg.  States her BP's were fine there since - 130-140/70-80.,",
  {'entities': [(40, 46, 'GPE'), (0, 7, 'PERSON')]}),
 ('Patient Mrs. Harshita presents for uri. Walks in Mumbai branch of Sassoon Hopital.  Overdue for follow-up by 3 months.Started last week.   Sinus pressure, post nasal drip , headache , ears blocked.,',
  {'entities': [(13, 21, 'PERSON')]}),
 ('Thyroid nodule : 2065, thyroid scan 2066 consistent with hot nodule and toxic multinodular goiter, saw Dr Rakesh Yadav, follow TSH.,',
  {'entities': [(106, 118, 'PERSON')]})]

In [7]:
## Training a blank spacy english model.

nlp = spacy.blank('en')  # create blank Language class
# create the built-in pipeline components and add them to the pipeline
# nlp.create_pipe works for built-ins that are registered with spaCy
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner, last=True)

# add labels
for _, annotations in tqdm(training_data):
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

# get names of other pipes to disable them during training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    for itn in range(20):
        print("Statring iteration " + str(itn))
        random.shuffle(training_data)
        losses = {}
        for text, annotations in training_data:
            nlp.update(
                [text],  # batch of texts
                [annotations],  # batch of annotations
                drop=0.2,  # dropout - make it harder to memorise data
                sgd=optimizer,  # callable to update weights
                losses=losses)
        print(losses)

100%|████████████████████████████████████████████████████████████████████████████| 446/446 [00:00<00:00, 226829.10it/s]


Statring iteration 0
{'ner': 941.1132381892475}
Statring iteration 1
{'ner': 400.40759329843576}
Statring iteration 2
{'ner': 333.07881598794944}
Statring iteration 3
{'ner': 314.61584145450774}
Statring iteration 4
{'ner': 232.70255729767442}
Statring iteration 5
{'ner': 443.7891955251608}
Statring iteration 6
{'ner': 250.67806476251386}
Statring iteration 7
{'ner': 250.18370509213162}
Statring iteration 8
{'ner': 199.82355202329992}
Statring iteration 9
{'ner': 183.85024864078042}
Statring iteration 10
{'ner': 192.7955765401986}
Statring iteration 11
{'ner': 172.1267300866909}
Statring iteration 12
{'ner': 175.66922857734937}
Statring iteration 13
{'ner': 271.3620839865394}
Statring iteration 14
{'ner': 171.35827342153075}
Statring iteration 15
{'ner': 195.02814599917966}
Statring iteration 16
{'ner': 173.34813549945895}
Statring iteration 17
{'ner': 156.22377368169631}
Statring iteration 18
{'ner': 116.24925079307798}
Statring iteration 19
{'ner': 164.73382519634256}


In [8]:
## Training pre trained spacy model.

nlp2 = spacy.load('en_core_web_sm')  # create blank Language class
# create the built-in pipeline components and add them to the pipeline
# nlp.create_pipe works for built-ins that are registered with spaCy
if 'ner' not in nlp2.pipe_names:
    ner = nlp2.create_pipe('ner')
    nlp2.add_pipe(ner, last=True)

# add labels
for _, annotations in tqdm(training_data):
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

# get names of other pipes to disable them during training
other_pipes = [pipe for pipe in nlp2.pipe_names if pipe != 'ner']
with nlp2.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp2.begin_training()
    for itn in range(20):
        print("Statring iteration " + str(itn))
        random.shuffle(training_data)
        losses = {}
        for text, annotations in training_data:
            nlp2.update(
                [text],  # batch of texts
                [annotations],  # batch of annotations
                drop=0.2,  # dropout - make it harder to memorise data
                sgd=optimizer,  # callable to update weights
                losses=losses)
        print(losses)

100%|████████████████████████████████████████████████████████████████████████████| 446/446 [00:00<00:00, 446351.61it/s]


Statring iteration 0
{'ner': 477.690066402281}
Statring iteration 1
{'ner': 258.81002657082036}
Statring iteration 2
{'ner': 226.30169552253994}
Statring iteration 3
{'ner': 201.39478648757986}
Statring iteration 4
{'ner': 146.3892170912794}
Statring iteration 5
{'ner': 160.1190894002787}
Statring iteration 6
{'ner': 147.76230418976567}
Statring iteration 7
{'ner': 141.16188566155577}
Statring iteration 8
{'ner': 160.5881912364581}
Statring iteration 9
{'ner': 163.31768586617036}
Statring iteration 10
{'ner': 151.75054687272052}
Statring iteration 11
{'ner': 124.76346305919688}
Statring iteration 12
{'ner': 137.88316379811113}
Statring iteration 13
{'ner': 131.0023602691603}
Statring iteration 14
{'ner': 119.55063925630422}
Statring iteration 15
{'ner': 123.80524862849335}
Statring iteration 16
{'ner': 114.04468242458584}
Statring iteration 17
{'ner': 82.99532781608653}
Statring iteration 18
{'ner': 98.80464153789157}
Statring iteration 19
{'ner': 86.49503042335417}


In [26]:
# Saving the trained model to disk.
nlp2.to_disk('trained_spacy_model2')